# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [5]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [6]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [7]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [8]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [9]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [10]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

In [ ]:
#Joined Both dataframes to later filter our requirements

In [14]:
joined_df = patents.join(citations,patents.PATENT ==  citations.CITING,"inner").cache()

In [19]:
joined_df.show(1)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|  CITED|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|3858258| 1975| 5485|   1974|     US|     CA|    null|      1|     6|     7|  5|    59|    2|       7|     0.0| 0.2449|    null| 20.1429|    52.5|    null|    null|    null|    null|3858258|1331793|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
only 

In [ ]:
#Filtering out the non null and non NAN feilds

In [24]:
joined_df.filter((joined_df.POSTATE!="null") & (joined_df.POSTATE!="")).show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|  CITED|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-------+
|3858258| 1975| 5485|   1974|     US|     CA|    null|      1|     6|     7|  5|    59|    2|       7|     0.0| 0.2449|    null| 20.1429|    52.5|    null|    null|    null|    null|3858258|1331793|
|3858258| 1975| 5485|   1974|     US|     CA|    null|      1|     6|     7|  5|    59|    2|       7|     0.0| 0.2449|    null| 20.1429|    52.5|    null|    null|    null|    null|3858258|1540798|
|3858

In [ ]:
#Dropping un-necessary columns and renaming postate to citing state

In [32]:
joined_df.filter((joined_df.POSTATE!="null") & (joined_df.POSTATE!=""))\
.withColumnRenamed('POSTATE','CITING_STATE').select('CITED', 'CITING_STATE', 'CITING').show(5)



+-------+------------+-------+
|  CITED|CITING_STATE| CITING|
+-------+------------+-------+
|1331793|          CA|3858258|
|1540798|          CA|3858258|
| 957631|          IN|3858560|
|3675252|          MT|3858597|
|3815160|          MT|3858597|
+-------+------------+-------+
only showing top 5 rows



In [ ]:
#Second Join to now get the cited columns postate. Followed by renaming and selecting useful columns

In [50]:
joined_df2 = joined_df.filter((joined_df.POSTATE!="null") & (joined_df.POSTATE!=""))\
.withColumnRenamed('POSTATE','CITING_STATE').select('CITED', 'CITING_STATE', 'CITING')\
.join(patents ,patents.PATENT ==  joined_df.CITED,"inner").cache()






In [51]:
joined_df2.filter((joined_df2.POSTATE!="null") & (joined_df2.POSTATE!=""))\
.withColumnRenamed('POSTATE','CITED_STATE').select('CITED', 'CITING_STATE', 'CITING','CITED_STATE').show(5)

+-------+------------+-------+-----------+
|  CITED|CITING_STATE| CITING|CITED_STATE|
+-------+------------+-------+-----------+
|3071753|          NJ|4496943|         MN|
|3071753|          TN|4345315|         MN|
|3071753|          IL|4120573|         MN|
|3071753|          NJ|3949375|         MN|
|3071753|          NY|4271479|         MN|
+-------+------------+-------+-----------+
only showing top 5 rows



In [ ]:
#Final match to get same state citations followed by grouping and counting. Dropped the columns that were not required

In [69]:
joined_df2 = joined_df2.filter((joined_df2.POSTATE!="null") & (joined_df2.POSTATE!=""))\
.withColumnRenamed('POSTATE','CITED_STATE').filter(joined_df2.CITING_STATE == joined_df2.POSTATE)\
.groupBy("CITING").count().withColumnRenamed('count','SAME_STATE')
                                                                                                                                                                     

In [71]:
final_df = patents.join(joined_df2 ,patents.PATENT ==  joined_df2.CITING,"inner").drop("CITING").sort(col("SAME_STATE").desc())

In [72]:
final_df.show(12)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 